<a href="https://colab.research.google.com/github/tayfununal/continuous_CNN/blob/main/firs_crossentropy_then_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import mnist
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
# Downloading dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("X_train : ",X_train.shape, " - type : ",type(X_train))
print("y_train : ",y_train.shape)
print("X_test : ",X_test.shape)
print("y_test : ",y_test.shape)

In [ ]:
# Herbir sınıftan 1000 veri alarak toplam veri kümesini 10 000 yaptık.

x = X_train[np.where(y_train==0)[0][:1000]]
y = y_train[np.where(y_train==0)[0][:1000]]

x_t =  X_test[np.where(y_test==0)[0][:200]]
y_t = y_test[np.where(y_test==0)[0][:200]]

for i in range(1,10):
  x_ek = X_train[np.where(y_train==i)[0][:1000]]
  x = np.concatenate((x, x_ek))

  y_ek = y_train[np.where(y_train==i)[0][:1000]]
  y = np.concatenate((y, y_ek))

  x_t_ek = X_test[np.where(y_test==i)[0][:200]]
  x_t = np.concatenate((x_t, x_t_ek))

  y_t_ek = y_test[np.where(y_test==i)[0][:200]]
  y_t = np.concatenate((y_t, y_t_ek))

print("Shape of x :", x.shape)
print("Shape of y :", y.shape)
print("Shape of x_t :", x_t.shape)
print("Shape of y_t :", y_t.shape)

In [ ]:
x = x.reshape(x.shape[0], 28, 28, 1)
x_t = x_t.reshape(x_t.shape[0], 28, 28, 1)

x = x.astype('float32')
x_t = x_t.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
x /= 255
x_t /= 255

print("Shape of x :", x.shape)
print("Shape of x_t :", x_t.shape)

In [ ]:
# Adding epsilon
tf.random.set_seed(8)
epsilon = tf.random.normal((10000,28,28,1), mean=0, stddev= 0.00001)
epsilon_test = tf.random.normal((2000,28,28,1), mean=0, stddev= 0.00001)

X_train_with_epsilon = tf.math.add(x, epsilon)
X_test_with_epsilon = tf.math.add(x_t, epsilon_test)

print('x shape:', X_train_with_epsilon.shape)
print('Number of images in x', X_train_with_epsilon.shape[0])
print('Number of images in x_t', X_test_with_epsilon.shape[0])

In [ ]:
# Making y_train and y_test to categorical
y = to_categorical(y)
y_t = to_categorical(y_t)

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

feature_input = Input(shape=(28,28))

hidden = Flatten()(feature_input)
hidden = Dense(512, activation="relu", name="one")(hidden)
hidden = Dense(512, activation="relu")(hidden)
hidden = Dense(256, activation="relu")(hidden)
hidden = Dense(256, activation="relu")(hidden)
hidden = Dense(128, activation="relu")(hidden)

output = Dense(10, activation='softmax')(hidden)

model = Model(inputs=feature_input , outputs=output)
model.summary()

In [ ]:
# Culculating Custom Loss
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x, y, epochs=50, verbose=1, batch_size=10000)

In [ ]:
batch_size = 10000
lamdas = 1.2
import keras.backend as K
class CustomMSE(tf.keras.losses.Loss):

    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        # custom cross_entropy = -1/batch_size * tf.reduce_sum(y_true * tf.math.log(y_pred + 10e-15))
        cross_entropy_loss = CategoricalCrossentropy()(y_true, y_pred)
        reg = lamdas * (1/batch_size) * tf.reduce_sum(abs(y_pred - model(X_train_with_epsilon)))

        return cross_entropy_loss + reg

In [ ]:
# Culculating Custom Loss
model.compile(optimizer="Adam", loss=CustomMSE(), metrics=["accuracy"])
history = model.fit(x, y, epochs=50, verbose=1, batch_size=10000, shuffle=True)

In [ ]:
# model.evaluate(x_t,  y_t, batch_size=2000, verbose=1)  -- reg'de model(X_train_with_epsilon) 10 000 boyutlu olduğundan sıkıntı çıkarıyor çünkü burada y_pred 2000
y_test = to_categorical(y_test)
model.evaluate(X_test,  y_test, batch_size=10000, verbose=1)

In [ ]:
predictions = model.predict(X_test)
print(predictions[0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 12, figsize=(30, 14))  # 4 = row , 12= column , 30 = horizontal size , 14 = vertical size

for i in range(0,4):
  for j in range(0,12):
    random_int = np.random.randint(1,2000,1)
    ax[i][j].imshow(X_test[random_int].reshape((28,28)), cmap=plt.cm.binary)
    ax[i][j].set_title("Value : {} \nPredicted Value : {}".format(int(np.where(y_test[random_int])[1]),np.argmax(predictions[random_int])))

plt.show()